In [2]:
import fnv
import fnv.reduce
import fnv.file  
import numpy as np
import seaborn as sns
import datetime as dt
import time
import pandas as pd
from tkinter import filedialog
from tkinter import *
import os
import matplotlib.pyplot as plt
import glob
import h5py
import cv2
from guppy import hpy
import gc
import shlex
import pipes
from subprocess import check_call
from sortedcontainers import SortedSet

gc.collect()

h5py.run_tests()

...........s...........................................x...........................s.....s......................................x....................................sC:\Users\cloftus\Anaconda3\lib\site-packages\h5py\_hl\files.py:258: H5pyDeprecationWarning: File.fid has been deprecated. Use File.id instead.
  "Use File.id instead.", H5pyDeprecationWarning)
...s......ss.ss...............................................................................ss...................ssssss.....................................................................x....x.........................x......x..................................................ssss...................s........................................
----------------------------------------------------------------------
Ran 509 tests in 1.614s

OK (skipped=22, expected failures=6)


<unittest.runner.TextTestResult run=509 errors=0 failures=0>

In [3]:
os.chdir('C:/Users/cloftus/Desktop/')

In [4]:
options = np.array([000000, 200000, 400000, 600000, 800000, 1000000])

def round_to_fifth(time_):
    
    new = options[ abs(time_.microsecond - options).argmin() ]
    
    if new != 1000000:
        new_time = time_.replace(microsecond = new)
    else:
        temp_time = time_ + dt.timedelta(seconds = 1)
        new_time = temp_time.replace(microsecond = 000000)
    
    return(new_time)

def time_correct( _timestamp_ ):
    if drive == 'D:/thermal_data/':
        _timestamp_ = _timestamp_ - time_offset_D
    if drive == 'F:/':
        _timestamp_ = _timestamp_ - time_offset_F
        
    return(_timestamp_)

In [5]:
options = np.array([000000, 200000, 400000, 600000, 800000, 1000000])


def round_to( time_):
    
    new = options[ abs( np.array( int( time_[ 6: ] ) ) - options).argmin() ]
    
    if new != 1000000:
        
        if new == 0:
            
            new_time = '000000'
            
        else:
            
            new_time = str( new )
        
        return( time_[ : 6 ] + new_time )
    else:
        
        new_time = '000000'
        
        return( time_[ : 5 ] + str( int( time_[ 5 ] ) + 1) + new_time )
    
    



In [6]:
# root = Tk()
# root.filename =  filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"),("all files","*.*")))
# print (root.filename)

In [7]:
datelist = [ d.date() for d in pd.date_range(start = pd.to_datetime('2019-08-11'), end = pd.to_datetime('2019-08-11')).to_pydatetime()]
print(datelist)

[datetime.date(2019, 8, 11)]


In [ ]:
os.chdir( 'C:/Users/cloftus/Documents/thermal_annotations/thermal_annotations_2020_10_12_csvs_special' )

zoom = 5

files = os.listdir()

drive = 'E:/'

#prev_file_name = 'blah'

for file in files:
    
    split_file = file.split( '_' )

    if len (split_file ) > 7:

        date, time = split_file[ 2 ], split_file[ 3 ]

        date_dt = dt.datetime.strptime( date, '%Y%m%d')

        if int( time.lstrip('0') ) < 90000000000:

            date_dt = date_dt - dt.timedelta( days = 1)

    else:

        if split_file[ 1 ] == '1' :

            date = split_file[ 2 ]

            date_dt = dt.datetime.strptime( date, '%Y%m%d')


        else:

            date, time = split_file[ 1 ], split_file[ 2 ]


            date_dt = dt.datetime.strptime( date, '%Y%m%d')

            if int( time.lstrip('0') ) < 90000000000:

                date_dt = date_dt - dt.timedelta( days = 1)


    curr_date = dt.datetime.strftime( date_dt, '%Y%m%d')
        
    os.chdir( drive + curr_date )

    file_name = glob.glob('*.seq')
    
    print( file_name, flush = True )

    if file_name[0] != prev_file_name:
        
        im = fnv.file.ImagerFile(file_name[0])
        
        print( 'got video')
        
        times = []

        for i in range( im.num_frames ):

            im.get_frame( i )

            temp_time = im.frame_info[0]['value'][4:]

            times.append( temp_time )

        
        prev_file_name = file_name[0]
        
        times = list( map( lambda x: x.replace( ".", "").replace( ":",""), times ) ) 
        
        print( 'finished with times' )
       
    
    
    if len (split_file ) > 7 :
        
        times = np.array( list( map( round_to, times ) ) )
        
        in_folder = dt.datetime.strftime( date_dt, '%Y%m%d')
        
        vid_name = split_file[ 1 ] + '_' + split_file[ 2 ] + '_' + split_file[ 3 ]
    
        os.chdir( 'C:/Users/cloftus/Documents/thermal_outputs/annotation_bursts/' + in_folder + '/' + vid_name + '/' )
    
        txt_name = vid_name + '.txt'
            
    else:
            
        if( split_file[ 1 ] == '1' ):

            times = np.array( list( map( round_to, times ) ) )

            in_folder = dt.datetime.strftime( date_dt, '%Y%m%d')

            vid_name = split_file[ 1 ] + '_' + split_file[ 2 ]

            os.chdir( 'C:/Users/cloftus/Desktop/Results/' + vid_name + '/' )

            txt_name = vid_name + '.txt'

        else:
            
            times = np.array( times )
            
            in_temp_tiff = dt.datetime.strftime( date_dt, '%Y-%m-%d')

            os.chdir( 'D:/temp_tiffs/' + in_temp_tiff + '/')

            txt_name = date + '_' + time + '.txt'

                    
    txt_times = []
    
    txt_file = open(txt_name, mode = 'r', encoding = 'utf-8-sig')
    lines = txt_file.readlines()
    
    if len (split_file ) > 7:
        
        for line in lines:

            txt_times.append( line.split( '.' )[ 0 ].split('_')[1] )
    else:
        
        for line in lines:

            txt_times.append( line.split( '.' )[ 0 ].split('_')[2] )
            
    os.chdir( 'C:/Users/cloftus/Documents/thermal_annotations/thermal_annotations_2020_10_12_csvs_special' )

    annos = pd.read_csv( file, header = 0 )

    frame_nums = SortedSet( annos[ 'frame_count' ] )
    
    frame_nums = np.array( list( map( int, frame_nums ) ) ) 
        
    final_times_ls = []
    
    for k in frame_nums:
        
        final_times_ls.append( txt_times[ k ] )
    
    os.chdir( 'F:/thermal_annotation_frames/' )
    
    write_path = file.split( '.' )[ 0 ] + '/'

    if not os.path.isdir( write_path ):
        
        os.makedirs( write_path )
    
    os.chdir( write_path )
        
    im.get_frame(0)
    
    day1 = int( im.frame_info[ 0 ]['value'][ :3 ] )

    final = np.array(im.final, copy=False).reshape((im.height, im.width))

    width = final.shape[1] / (100 / zoom)

    height = final.shape[0] / (100 / zoom)

    fig = plt.figure( figsize = ( width, height ) )  

    for j in range( len( final_times_ls ) ):
        
        now_time = final_times_ls[ j ]
        
        if( len( np.where( times == now_time )[ 0 ] ) == 0 ):
            
            print( "no matching frame for this vid: " + str( file_name ) + ', at this time: ' + str( now_time ) )
            
        else:
        
        
            needed_frame = np.where( times == now_time )[ 0 ][ 0 ]

            im.get_frame( needed_frame )

            day2 = int( im.frame_info[ 0 ]['value'][ :3 ] )

            day_diff = day2 - day1

            timestamp = f'{date_dt.year:04}-' + f'{date_dt.month:02}-' + f'{(date_dt.day+day_diff):02} ' + im.frame_info[0]['value'][4:]

            timestamp = dt.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')


            final = np.array(im.final, copy=False).reshape((im.height, im.width))

            final1 = final - final.mean()

            sns.heatmap(final1, vmin = -np.std(final1), vmax = final1.max(), xticklabels = False, yticklabels = False, cbar = False)

            if final1.shape[1] > 1000:

                plt.xlabel(  timestamp.strftime("%Y-%m-%d %H:%M:%S.%f") , fontsize = 55*zoom / 5 )

            else:

                plt.xlabel(  timestamp.strftime("%Y-%m-%d %H:%M:%S.%f") , fontsize = 25*zoom / 5 )


            plt.savefig( f'{frame_nums[ j ]:06}_' + timestamp.strftime("%Y%m%d_%H%M%S%f") + '.tiff' , bbox_inches = 'tight')

            fig.clear()

            del day2, day_diff, final, final1

            gc.collect()



['20190731_18_00_00-212_15_00_08_934.seq']
['20190801_17_06_00-213_14_06_02_088.seq']
got video
finished with times
no matching frame for this vid: ['20190801_17_06_00-213_14_06_02_088.seq'], at this time: 165750000000
['20190801_17_06_00-213_14_06_02_088.seq']
['20190803_16_30_00-215_13_30_06_187.seq']
got video
finished with times
['20190803_16_30_00-215_13_30_06_187.seq']
['20190803_16_30_00-215_13_30_06_187.seq']
['20190803_16_30_00-215_13_30_06_187.seq']


In [ ]:
os.chdir( 'C:/Users/cloftus/Documents/thermal_annotations/thermal_annotations_2020_10_12_csvs_special/' )

zoom = 5

files = os.listdir()

drive = 'E:/'

prev_file_name = 'blah'

for file in files:
    
    split_file = file.split( '_' )

    if len (split_file ) > 7:

        date, time = split_file[ 2 ], split_file[ 3 ]

        date_dt = dt.datetime.strptime( date, '%Y%m%d')

        if int( time.lstrip('0') ) < 90000000000:

            date_dt = date_dt - dt.timedelta( days = 1)

    else:

        if split_file[ 1 ] == '1' :

            date = split_file[ 2 ]

            date_dt = dt.datetime.strptime( date, '%Y%m%d')


        else:

            date, time = split_file[ 1 ], split_file[ 2 ]


            date_dt = dt.datetime.strptime( date, '%Y%m%d')

            if int( time.lstrip('0') ) < 90000000000:

                date_dt = date_dt - dt.timedelta( days = 1)


    curr_date = dt.datetime.strftime( date_dt, '%Y%m%d')
        
    os.chdir( drive + curr_date )

    file_name = glob.glob('*.seq')
    
    print( file_name )

    if file_name[0] != prev_file_name:
        
        im = fnv.file.ImagerFile(file_name[0])
        
        print( 'got video')
        
        times = []

        for i in range( im.num_frames ):

            im.get_frame( i )

            temp_time = im.frame_info[0]['value'][4:]
            
            temp_time = temp_time.replace( ".", "").replace( ":","")
            
            temp_time = round_to( temp_time )

            times.append( temp_time )

        
        prev_file_name = file_name[0]
                        
        print( 'finished with times' )
       
    
    
    if len (split_file ) > 7 :
        
        
        in_folder = dt.datetime.strftime( date_dt, '%Y%m%d')
        
        vid_name = split_file[ 1 ] + '_' + split_file[ 2 ] + '_' + split_file[ 3 ]
    
        os.chdir( 'C:/Users/cloftus/Documents/thermal_outputs/annotation_bursts/' + in_folder + '/' + vid_name + '/' )
    
        txt_name = vid_name + '.txt'
            
    else:
            
        if( split_file[ 1 ] == '1' ):

            in_folder = dt.datetime.strftime( date_dt, '%Y%m%d')

            vid_name = split_file[ 1 ] + '_' + split_file[ 2 ]

            os.chdir( 'C:/Users/cloftus/Desktop/Results/' + vid_name + '/' )

            txt_name = vid_name + '.txt'

        else:
                    

            in_temp_tiff = dt.datetime.strftime( date_dt, '%Y-%m-%d')

            os.chdir( 'D:/temp_tiffs/' + in_temp_tiff + '/')

            txt_name = date + '_' + time + '.txt'

                    
    txt_times = []
    
    txt_file = open(txt_name, mode = 'r', encoding = 'utf-8-sig')
    lines = txt_file.readlines()
    
    for line in lines:
        
        txt_times.append( line.split( '.' )[ 0 ].split('_')[2] )
            
            
    os.chdir( 'C:/Users/cloftus/Documents/thermal_annotations/thermal_annotations_2020_10_12_csvs_special/' )

    annos = pd.read_csv( file, header = 0 )

    frame_nums = SortedSet( annos[ 'frame_count' ] )
    
    frame_nums = np.array( list( map( int, frame_nums ) ) ) 
        
    final_times_ls = []
    
    for k in frame_nums:
        
        final_times_ls.append( txt_times[ k ] )
    
    os.chdir( 'F:/thermal_annotation_frames/' )
    
    write_path = file.split( '.' )[ 0 ] + '/'

    if not os.path.isdir( write_path ):
        
        os.makedirs( write_path )
    
    os.chdir( write_path )
        
    im.get_frame(0)
    
    day1 = int( im.frame_info[ 0 ]['value'][ :3 ] )

    final = np.array(im.final, copy=False).reshape((im.height, im.width))

    width = final.shape[1] / (100 / zoom)

    height = final.shape[0] / (100 / zoom)

    fig = plt.figure( figsize = ( width, height ) )  

    for j in range( len( final_times_ls ) ):
        
        now_time = final_times_ls[ j ]
        
        needed_frame = np.where( times == now_time )[ 0 ]
        
        im.get_frame( needed_frame )
     
        day2 = int( im.frame_info[ 0 ]['value'][ :3 ] )

        day_diff = day2 - day1
        
        timestamp = f'{date_dt.year:04}-' + f'{date_dt.month:02}-' + f'{(date_dt.day+day_diff):02} ' + im.frame_info[0]['value'][4:]

        timestamp = dt.datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S.%f')
        

        final = np.array(im.final, copy=False).reshape((im.height, im.width))

        final1 = final - final.mean()

        sns.heatmap(final1, vmin = -np.std(final1), vmax = final1.max(), xticklabels = False, yticklabels = False, cbar = False)

        if final1.shape[1] > 1000:

            plt.xlabel(  timestamp.strftime("%Y-%m-%d %H:%M:%S.%f") , fontsize = 55*zoom / 5 )

        else:

            plt.xlabel(  timestamp.strftime("%Y-%m-%d %H:%M:%S.%f") , fontsize = 25*zoom / 5 )


        plt.savefig( f'{frame_nums[ j ]:06}_' + timestamp.strftime("%Y%m%d_%H%M%S%f") + '.tiff' , bbox_inches = 'tight')

        fig.clear()

        del day2, day_diff, final, final1

        gc.collect()

